In [1]:
!pip install pandas
!pip install tensorflow
!pip install keras

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
print("hi")

hi


In [4]:
train_dir= 'C:\\Users\\samba\\Downloads\\archive\\real_and_fake_face\\train_sample'
valid_dir= 'C:\\Users\\samba\\Downloads\\archive\\real_and_fake_face\\Test_sample'
img_width,img_height = 600, 600

In [5]:
print("hi")
conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(img_width,img_height,3))
conv_base.summary()

hi
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 600, 600, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 600, 600, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 600, 600, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 300, 300, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 300, 300, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 300, 300, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 150, 150, 128)     0  

In [12]:
# Extract features
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 18, 18, 512))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='binary')
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 240)
validation_features, validation_labels = extract_features(valid_dir, 60)

Found 240 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [13]:
#train_features.Flatten()
test_features=validation_features
test_labels=validation_labels

In [14]:
train_flattened = train_features.flatten().reshape(train_features.shape[0],train_features.shape[1]*train_features.shape[2]*train_features.shape[3])
test_flattened = test_features.flatten().reshape(test_features.shape[0],test_features.shape[1]*test_features.shape[2]*test_features.shape[3])
np.savetxt("C:\\Users\\samba\\Downloads\\archive\\train_sample_features.csv",train_flattened, delimiter=",")
np.savetxt("C:\\Users\\samba\\Downloads\\archive\\train_sample_labels.csv",train_labels, delimiter=",")
np.savetxt("C:\\Users\\samba\\Downloads\\archive\\test_sample_features.csv",test_flattened, delimiter=",")
np.savetxt("C:\\Users\\samba\\Downloads\\archive\\test_sample_labels.csv",test_labels, delimiter=",")

In [ ]:
epochs = 150

model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=(7,7,512)))
model.add(Dense(6, activation='softmax'))
model.summary()

In [ ]:
from keras.callbacks.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

# Compile model
from keras.optimizers import Adam
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

# Train model
history = model.fit(train_features, train_labels,
                    epochs=epochs,
                    batch_size=batch_size, 
                    callbacks=[checkpoint],
                    validation_data=(validation_features, validation_labels))

In [ ]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])